Upload a CSV to the same directory as this notebook that has at least a list of IDs with associated text that needs to be analysed. Then, in the cell below change the `input`, `id_column`, and `text_column` as appropriate and execute.

In [ ]:
input = "nectar-use-cases.csv"
id_column = 'project_name'
text_column = 'use_case'

# Don't edit below this line

import csv
import requests
import urllib
import json
import os

output = "mentions-" + input 
api_endpoint = 'http://software-mentions:8060/service/processSoftwareText'

Execute the below cell to start the analysis. It will take a while, especially if you don't have GPU acceleration!

The first analysis in particular will take a while, subsequent analyses should be faster as software-mentions has loaded stuff it needs to load.

In [ ]:
with open(input) as inputfile:
    inputreader = csv.DictReader(inputfile)
    with open(output, 'w') as outputfile:
        logwriter = csv.writer(outputfile)
        logwriter.writerow([id_column, 'num_mentions', 'mentions_json'])
        for row in inputreader:
            use_case = row[text_column]
            print('Processing ' + row[id_column], end = '...')
            text = row[text_column]
            data = { 'disambiguate': '1', 'text': text }
            response = requests.request('POST', api_endpoint, data = data)
            if ( response.status_code == 200 ):
                mentions_json = json.loads(response.text)
                num_mentions = len(mentions_json['mentions'])
                print('Success! ' + str(num_mentions) + ' mentions.')
                logwriter.writerow([row[id_column], num_mentions, json.dumps(mentions_json, indent=4)])
            else:
                print("Error! " + response.status_text)
        